In [50]:
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('data/processed/heart_risk.csv')
irrelevant = ['id']
df = df.drop(irrelevant, axis=1)

In [33]:
df.corr()['ami_risk'].sort_values()

ejection_frac_ex      -1.000000
heart_rate_peak       -0.400459
ex_duration           -0.311104
rldv5e                -0.179731
chol                  -0.170667
blood_press_s_peak    -0.134437
heart_rate_rest       -0.125558
st_dep_measured_at     0.004065
rldv5                  0.005515
blood_press_d_rest     0.013144
ejection_frac_rest     0.037608
fasting_blood_sugar    0.079734
blood_press_d_peak     0.083487
smoker_cigs_per_day    0.110257
blood_press_s_rest     0.144814
smoker_years           0.172921
met                    0.205721
age                    0.230523
st_dep                 0.386107
vessels_colored        0.497286
ami_risk               1.000000
pncaden                     NaN
Name: ami_risk, dtype: float64

In [36]:
df.isna().mean().sort_values()

age                     0.000000
sex                     0.000000
source                  0.000000
ami_risk                0.000000
cp_type                 0.000000
rest_ecg                0.001653
chol                    0.011570
cath_date               0.018182
ex_duration             0.089256
ex_angina               0.089256
heart_rate_peak         0.089256
ekg_date                0.089256
heart_rate_rest         0.090909
ex_hypotension          0.092562
blood_press_d_rest      0.095868
blood_press_s_rest      0.095868
blood_press_s_peak      0.102479
ccblocker               0.102479
st_dep                  0.102479
blood_press_d_peak      0.102479
betablocker             0.105785
nitrate                 0.105785
digitalis               0.110744
diuretic                0.133884
fasting_blood_sugar     0.135537
met                     0.170248
ex_protocol             0.170248
st_slope_peak           0.196694
smoker_cigs_per_day     0.209917
heart_disease_family    0.213223
smoker_yea

In [40]:
df[(df.smoker == 'yes') | (df.smoker_cigs_per_day > 0) | (df.smoker_years > 0)][['smoker', 'smoker_cigs_per_day', 'smoker_years']]


,smoker,smoker_cigs_per_day,smoker_years
0,NaN,50.0,20.0
1,NaN,40.0,40.0
2,NaN,20.0,35.0
5,NaN,20.0,20.0
9,NaN,20.0,25.0
...,...,...,...
572,yes,20.0,NaN
574,yes,20.0,NaN
577,yes,20.0,NaN
582,yes,NaN,NaN


In [48]:
df.age

0      63
1      67
2      67
3      37
4      41
       ..
600    54
601    56
602    56
603    54
604    66
Name: age, Length: 605, dtype: int64

In [49]:
from framingham import FraminghamModel
fram = FraminghamModel()
fram_results = df.apply(fram.predict, axis=1)

fore assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' referenced before assignment
local variable 'age_score' refer

In [46]:
fram_results

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
600   NaN
601   NaN
602   NaN
603   NaN
604   NaN
Length: 605, dtype: float64

In [37]:
df.drop(df.isna().mean()[df.isna().mean() >= .75].index, axis=1)

,age,sex,cp_loc,cp_exertion,cp_rest_relieve,cp_type,blood_press_s_rest,chol,smoker,smoker_cigs_per_day,...,st_dep,st_slope_peak,rldv5,rldv5e,vessels_colored,thalassemia,ami_risk,source,ekg_date,cath_date
0,63,M,NaN,NaN,NaN,typical_angina,145.0,233.0,NaN,50.0,...,2.3,down,NaN,172.0,0.0,fixed_defect,0,cleveland_minus_corrupted,1981-02-03,1981-02-16
1,67,M,NaN,NaN,NaN,no_pain,160.0,286.0,NaN,40.0,...,1.5,flat,NaN,185.0,3.0,normal,2,cleveland_minus_corrupted,1981-03-05,1981-02-05
2,67,M,NaN,NaN,NaN,no_pain,120.0,229.0,NaN,20.0,...,2.6,flat,NaN,150.0,2.0,reversible,1,cleveland_minus_corrupted,1981-02-19,1981-02-20
3,37,M,NaN,NaN,NaN,non_anginal,130.0,250.0,NaN,0.0,...,3.5,down,NaN,167.0,0.0,normal,0,cleveland_minus_corrupted,1981-02-13,1981-02-04
4,41,F,NaN,NaN,NaN,atypical_angina,130.0,204.0,NaN,0.0,...,1.4,up,NaN,40.0,0.0,normal,0,cleveland_minus_corrupted,1981-02-07,1981-02-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,54,M,substernal,yes,yes,no_pain,180.0,0.0,NaN,NaN,...,1.5,flat,NaN,NaN,NaN,reversible,1,switzerland,1985-05-06,1985-05-07
601,56,M,substernal,yes,yes,no_pain,125.0,0.0,yes,NaN,...,1.0,flat,NaN,NaN,NaN,reversible,3,switzerland,1985-06-19,1985-06-20
602,56,M,non_substernal,yes,yes,non_anginal,125.0,0.0,NaN,NaN,...,-2.0,flat,NaN,NaN,NaN,reversible,2,switzerland,1985-06-24,1985-06-25
603,54,M,substernal,yes,yes,no_pain,130.0,0.0,NaN,NaN,...,3.0,flat,NaN,NaN,NaN,reversible,3,switzerland,1985-07-03,1985-07-04
